In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
movies =[]
url=['https://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2018&p=.htm'
    ]

for i in range(2): 
    page = requests.get(url[i])
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    tr_elements = tr_elements[9:109]
    mov = []
    for xd in range(len(tr_elements)):
        for a in tr_elements[xd]:
            mov.append(a.text_content())
            try:
                url_mov = a.getchildren()[0].getchildren()[0].getchildren()[0].attrib['href']
                page2 = requests.get('https://www.boxofficemojo.com'+url_mov)
                doc2 = lh.fromstring(page2.content)
                tr_elements2 = doc2.xpath('//tr')
                mov.append(tr_elements2[7].getchildren()[0].getchildren()[0].text_content()) #distributor full name
                mov.append(tr_elements2[7].getchildren()[1].getchildren()[0].text_content()) #release date
                mov.append(tr_elements2[8].getchildren()[0].getchildren()[0].text_content()) # genre
                mov.append(tr_elements2[8].getchildren()[1].getchildren()[0].text_content()) #runtime
                mov.append(tr_elements2[9].getchildren()[0].getchildren()[0].text_content()) #rating
                mov.append(tr_elements2[9].getchildren()[1].getchildren()[0].text_content()) #budget
                if(tr_elements2[15].getchildren()[0].text_content().replace(u'\xa0', u'').replace('+','') == 'Foreign:'):
                    mov.append(tr_elements2[15].getchildren()[1].text_content().replace(u'\xa0', u'')) #rest world
                else:
                    mov.append('no dist')
            except: 
                pass #XD2
        movies.append(mov)
        mov = []

In [6]:
headers =['#','name','distributor_full_name','release_date','genre','runtime','rating','budget','rest_world','studio','total_gross','theaters_total','opening','theaters_opening','open_date','close_date',]
df = pd.DataFrame(movies,columns = headers)
df.head()
df.to_csv('movies.csv')